In [17]:
KDM5s = ["Q3UXZ9","Q38JA7","Q30DN6","Q62240","Q5F3R2","Q80Y84","A1YVX4","Q61T02","Q9VMJ7","P41228","P41229","Q5XUN4","Q23541","P29375","Q9UGL1","P41230","Q9BY66","Q6IQX0","P47156","Q8GUI6"]

In [22]:
#Try using Bio.Entrez
from Bio import Entrez
from Bio import SeqIO
Entrez.email = "david.brown3@ucsf.edu"  # Always tell NCBI who you are
handle = Entrez.einfo()
result = handle.read()
handle.close()

In [2]:
print(result)

<?xml version="1.0" encoding="UTF-8" ?>
<!DOCTYPE eInfoResult PUBLIC "-//NLM//DTD einfo 20130322//EN" "https://eutils.ncbi.nlm.nih.gov/eutils/dtd/20130322/einfo.dtd">
<eInfoResult>
<DbList>

	<DbName>pubmed</DbName>
	<DbName>protein</DbName>
	<DbName>nuccore</DbName>
	<DbName>ipg</DbName>
	<DbName>nucleotide</DbName>
	<DbName>nucgss</DbName>
	<DbName>nucest</DbName>
	<DbName>structure</DbName>
	<DbName>sparcle</DbName>
	<DbName>genome</DbName>
	<DbName>annotinfo</DbName>
	<DbName>assembly</DbName>
	<DbName>bioproject</DbName>
	<DbName>biosample</DbName>
	<DbName>blastdbinfo</DbName>
	<DbName>books</DbName>
	<DbName>cdd</DbName>
	<DbName>clinvar</DbName>
	<DbName>clone</DbName>
	<DbName>gap</DbName>
	<DbName>gapplus</DbName>
	<DbName>grasp</DbName>
	<DbName>dbvar</DbName>
	<DbName>gene</DbName>
	<DbName>gds</DbName>
	<DbName>geoprofiles</DbName>
	<DbName>homologene</DbName>
	<DbName>medgen</DbName>
	<DbName>mesh</DbName>
	<DbName>ncbisearch</DbName>
	<DbName>nlmcatalog</DbName>
	<DbName

In [3]:
handle = Entrez.einfo()
record = Entrez.read(handle)

In [4]:
record.keys()

[u'DbList']

In [5]:
record["DbList"]

['pubmed', 'protein', 'nuccore', 'ipg', 'nucleotide', 'nucgss', 'nucest', 'structure', 'sparcle', 'genome', 'annotinfo', 'assembly', 'bioproject', 'biosample', 'blastdbinfo', 'books', 'cdd', 'clinvar', 'clone', 'gap', 'gapplus', 'grasp', 'dbvar', 'gene', 'gds', 'geoprofiles', 'homologene', 'medgen', 'mesh', 'ncbisearch', 'nlmcatalog', 'omim', 'orgtrack', 'pmc', 'popset', 'probe', 'proteinclusters', 'pcassay', 'biosystems', 'pccompound', 'pcsubstance', 'pubmedhealth', 'seqannot', 'snp', 'sra', 'taxonomy', 'biocollections', 'unigene', 'gencoll', 'gtr']

In [6]:
handle = Entrez.esearch(db="protein", term="Q3UXZ9")
record = Entrez.read(handle)

In [7]:
#This give the NCBI id for the correct protein
print(record["IdList"])

['150383496']


In [8]:
record.attributes()

TypeError: 'dict' object is not callable

In [ ]:
print(record.attributes)

In [11]:
#For each of these databases, we can use EInfo again to obtain more information:
handle = Entrez.einfo(db="protein")
record = Entrez.read(handle)
record["DbInfo"]["Description"]

'Protein sequence record'

In [12]:
record["DbInfo"]["Count"]

'534686518'

Try record["DbInfo"].keys() for other information stored in this record. One of the most useful is a list of possible search fields for use with ESearch:

>>> for field in record["DbInfo"]["FieldList"]:
...     print("%(Name)s, %(FullName)s, %(Description)s" % field)
...
ALL, All Fields, All terms from all searchable fields
UID, UID, Unique number assigned to publication
FILT, Filter, Limits the records
TITL, Title, Words in title of publication
WORD, Text Word, Free text associated with publication
MESH, MeSH Terms, Medical Subject Headings assigned to publication
MAJR, MeSH Major Topic, MeSH terms of major importance to publication
AUTH, Author, Author(s) of publication
JOUR, Journal, Journal abbreviation of publication
AFFL, Affiliation, Author's institutional affiliation and address
...
That’s a long list, but indirectly this tells you that for the PubMed database, you can do things like Jones[AUTH] to search the author field, or Sanger[AFFL] to restrict to authors at the Sanger Centre. This can be very handy - especially if you are not so familiar with a particular database.

In [13]:
#What information is stored in the record?
record["DbInfo"].keys()

[u'Count',
 u'LastUpdate',
 u'MenuName',
 u'Description',
 u'LinkList',
 u'DbBuild',
 u'FieldList',
 u'DbName']

In [15]:
for field in record["DbInfo"]["FieldList"]:
    print("%(Name)s, %(FullName)s, %(Description)s" % field)

ALL, All Fields, All terms from all searchable fields
UID, UID, Unique number assigned to each sequence
FILT, Filter, Limits the records
WORD, Text Word, Free text associated with record
TITL, Title, Words in definition line
KYWD, Keyword, Nonstandardized terms provided by submitter
AUTH, Author, Author(s) of publication
JOUR, Journal, Journal abbreviation of publication
VOL, Volume, Volume number of publication
ISS, Issue, Issue number of publication
PAGE, Page Number, Page number(s) of publication
ORGN, Organism, Scientific and common names of organism, and all higher levels of taxonomy
ACCN, Accession, Accession number of sequence
PACC, Primary Accession, Does not include retired secondary accessions
GENE, Gene Name, Name of gene associated with sequence
PROT, Protein Name, Name of protein associated with sequence
ECNO, EC/RN Number, EC number for enzyme or CAS registry number
PDAT, Publication Date, Date sequence added to GenBank
MDAT, Modification Date, Date of last update
SUBS, S

EFetch: Downloading full records from Entrez

EFetch is what you use when you want to retrieve a full record from Entrez. This covers several possible databases, as described on the main EFetch Help page.

For most of their databases, the NCBI support several different file formats. Requesting a specific file format from Entrez using Bio.Entrez.efetch() requires specifying the rettype and/or retmode optional arguments. The different combinations are described for each database type on the pages linked to on NCBI efetch webpage (e.g. literature, sequences and taxonomy).

One common usage is downloading sequences in the FASTA or GenBank/GenPept plain text formats (which can then be parsed with Bio.SeqIO, see Sections 5.3.1 and 9.6). From the Cypripedioideae example above, we can download GenBank record EU490707 using Bio.Entrez.efetch:



In [16]:
#Cookbook example
handle = Entrez.efetch(db="nucleotide", id="EU490707", rettype="gb", retmode="text")
print(handle.read())

LOCUS       EU490707                1302 bp    DNA     linear   PLN 26-JUL-2016
DEFINITION  Selenipedium aequinoctiale maturase K (matK) gene, partial cds;
            chloroplast.
ACCESSION   EU490707
VERSION     EU490707.1
KEYWORDS    .
SOURCE      chloroplast Selenipedium aequinoctiale
  ORGANISM  Selenipedium aequinoctiale
            Eukaryota; Viridiplantae; Streptophyta; Embryophyta; Tracheophyta;
            Spermatophyta; Magnoliophyta; Liliopsida; Asparagales; Orchidaceae;
            Cypripedioideae; Selenipedium.
REFERENCE   1  (bases 1 to 1302)
  AUTHORS   Neubig,K.M., Whitten,W.M., Carlsward,B.S., Blanco,M.A., Endara,L.,
            Williams,N.H. and Moore,M.
  TITLE     Phylogenetic utility of ycf1 in orchids: a plastid gene more
            variable than matK
  JOURNAL   Plant Syst. Evol. 277 (1-2), 75-84 (2009)
REFERENCE   2  (bases 1 to 1302)
  AUTHORS   Neubig,K.M., Whitten,W.M., Carlsward,B.S., Blanco,M.A.,
            Endara,C.L., Williams,N.H. and Moore,M.J.
  TIT

In [18]:
#My KDM5s Protein list
handle = Entrez.efetch(db="protein", id=KDM5s[0], rettype="gb", retmode="text")
print(handle.read())

LOCUS       KDM5A_MOUSE             1690 aa            linear   ROD 28-MAR-2018
DEFINITION  RecName: Full=Lysine-specific demethylase 5A; AltName: Full=Histone
            demethylase JARID1A; AltName: Full=Jumonji/ARID domain-containing
            protein 1A; AltName: Full=Retinoblastoma-binding protein 2;
            Short=RBBP-2.
ACCESSION   Q3UXZ9
VERSION     Q3UXZ9.2
DBSOURCE    UniProtKB: locus KDM5A_MOUSE, accession Q3UXZ9;
            class: standard.
            extra accessions:Q3TM94,Q3UMI5,Q66JZ3
            created: Jun 26, 2007.
            sequence updated: Jun 26, 2007.
            annotation updated: Mar 28, 2018.
            xrefs: AC155720.7, AC078896.23, AK135085.1, BAE22414.1, AK144877.1,
            BAE26113.1, AK166055.1, BAE38548.1, BC080691.1, AAH80691.1,
            NP_666109.2
            xrefs (non-sequence databases): CCDS:CCDS51889.1,
            UniGene:Mm.404761, UniGene:Mm.463658, ProteinModelPortal:Q3UXZ9,
            SMR:Q3UXZ9, BioGrid:229572, IntAc

The arguments rettype="gb" and retmode="text" let us download this record in the GenBank format.

Note that until Easter 2009, the Entrez EFetch API let you use “genbank” as the return type, however the NCBI now insist on using the official return types of “gb” or “gbwithparts” (or “gp” for proteins) as described on online. Also note that until Feb 2012, the Entrez EFetch API would default to returning plain text files, but now defaults to XML.

Alternatively, you could for example use rettype="fasta" to get the Fasta-format; see the EFetch Sequences Help page for other options. Remember – the available formats depend on which database you are downloading from - see the main EFetch Help page.

If you fetch the record in one of the formats accepted by Bio.SeqIO (see Chapter 5), you could directly parse it into a SeqRecord:

In [19]:
#Try changing to 'gp' = No obvious difference
#My KDM5s Protein list
handle = Entrez.efetch(db="protein", id=KDM5s[0], rettype="gp", retmode="text")
print(handle.read())

LOCUS       KDM5A_MOUSE             1690 aa            linear   ROD 28-MAR-2018
DEFINITION  RecName: Full=Lysine-specific demethylase 5A; AltName: Full=Histone
            demethylase JARID1A; AltName: Full=Jumonji/ARID domain-containing
            protein 1A; AltName: Full=Retinoblastoma-binding protein 2;
            Short=RBBP-2.
ACCESSION   Q3UXZ9
VERSION     Q3UXZ9.2
DBSOURCE    UniProtKB: locus KDM5A_MOUSE, accession Q3UXZ9;
            class: standard.
            extra accessions:Q3TM94,Q3UMI5,Q66JZ3
            created: Jun 26, 2007.
            sequence updated: Jun 26, 2007.
            annotation updated: Mar 28, 2018.
            xrefs: AC155720.7, AC078896.23, AK135085.1, BAE22414.1, AK144877.1,
            BAE26113.1, AK166055.1, BAE38548.1, BC080691.1, AAH80691.1,
            NP_666109.2
            xrefs (non-sequence databases): CCDS:CCDS51889.1,
            UniGene:Mm.404761, UniGene:Mm.463658, ProteinModelPortal:Q3UXZ9,
            SMR:Q3UXZ9, BioGrid:229572, IntAc

In [20]:
#Try rettype="fasta" = Less informative but more concise.
#My KDM5s Protein list
handle = Entrez.efetch(db="protein", id=KDM5s[0], rettype="fasta", retmode="text")
print(handle.read())

>sp|Q3UXZ9.2|KDM5A_MOUSE RecName: Full=Lysine-specific demethylase 5A; AltName: Full=Histone demethylase JARID1A; AltName: Full=Jumonji/ARID domain-containing protein 1A; AltName: Full=Retinoblastoma-binding protein 2; Short=RBBP-2
MASVGPGGYAAEFVPPPECPVFEPSWEEFTDPLSFIGRIRPFAEKTGICKIRPPKDWQPPFACEVKTFRF
TPRVQRLNELEAMTRVRLDFLDQLAKFWELQGSTLKIPVVERKILDLYALSKIVASKGGFEIVTKEKKWS
KVGSRLGYLPGKGTGSLLKSHYERILYPYELFQSGVSLMGVQMPDLDLKEKVEAEVLSTDIQPSPERGTR
MNIPPKRTRRVKSQSDSGEVNRNTELKKLQIFGAGPKVVGLAVGAKDKEDEVTRRRKVTNRSDAFNMQMR
QRKGTLSVNFVDLYVCMFCGRGNNEDKLLLCDGCDDSYHTFCLLPPLPDVPKGDWRCPKCVAEECNKPRE
AFGFEQAVREYTLQSFGEMADNFKSDYFNMPVHMVPTELVEKEFWRLVSSIEEDVIVEYGADISSKDFGS
GFPKKDGQRKMLPEEEEYALSGWNLNNMPVLEQSVLAHINVDISGMKVPWLYVGMCFSSFCWHIEDHWSY
SINYLHWGEPKTWYGVPSHAAEQLEEVMRELAPELFESQPDLLHQLVTIMNPNVLMEHGVPVYRTNQCAG
EFVVTFPRAYHSGFNQGYNFAEAVNFCTADWLPIGRQCVNHYRRLRRHCVFSHEELIFKMAADPECLDVG
LAAMVCKELTLMTEEETRLRESVVQMGVVMSEEEVFELVPDDERQCSACRTTCFLSALTCSCNPERLVCL
YHPTDLCSCPMQNKCLRYRYPLEDLPSLLYGVKVRAQSYDTWVNRVTEALSASFNHKK

If you fetch the record in one of the formats accepted by Bio.SeqIO (see Chapter 5), you could directly parse it into a SeqRecord:


In [23]:
#Cookbook example
handle = Entrez.efetch(db="nucleotide", id="EU490707", rettype="gb", retmode="text")
record = SeqIO.read(handle, "genbank")
handle.close()
print(record.id)
print(record.name)
print(record.description)
print(len(record.features))
print(repr(record.seq))

EU490707.1
EU490707
Selenipedium aequinoctiale maturase K (matK) gene, partial cds; chloroplast
3
Seq('ATTTTTTACGAACCTGTGGAAATTTTTGGTTATGACAATAAATCTAGTTTAGTA...GAA', IUPACAmbiguousDNA())


In [28]:
#My version
handle = Entrez.efetch(db="protein", id=KDM5s[0], rettype="gb", retmode="text")
record = SeqIO.read(handle, "genbank")
handle.close()
print(record.id)
print(record.name)
print(record.description)
print(len(record.features))
print(repr(record.seq))

Q3UXZ9.2
KDM5A_MOUSE
RecName: Full=Lysine-specific demethylase 5A; AltName: Full=Histone demethylase JARID1A; AltName: Full=Jumonji/ARID domain-containing protein 1A; AltName: Full=Retinoblastoma-binding protein 2; Short=RBBP-2
55
Seq('MASVGPGGYAAEFVPPPECPVFEPSWEEFTDPLSFIGRIRPFAEKTGICKIRPP...ETS', IUPACProtein())


In [25]:
print(record.features)

[SeqFeature(FeatureLocation(ExactPosition(0), ExactPosition(1690)), type='source'), SeqFeature(FeatureLocation(ExactPosition(0), ExactPosition(1690)), type='gene'), SeqFeature(FeatureLocation(ExactPosition(0), ExactPosition(1690)), type='Protein'), SeqFeature(FeatureLocation(ExactPosition(0), ExactPosition(1690)), type='Region'), SeqFeature(FeatureLocation(ExactPosition(9), ExactPosition(10)), type='Region'), SeqFeature(FeatureLocation(ExactPosition(13), ExactPosition(14)), type='Region'), SeqFeature(FeatureLocation(ExactPosition(16), ExactPosition(17)), type='Region'), SeqFeature(FeatureLocation(ExactPosition(17), ExactPosition(59)), type='Region'), SeqFeature(FeatureLocation(ExactPosition(18), ExactPosition(60)), type='Region'), SeqFeature(FeatureLocation(ExactPosition(23), ExactPosition(24)), type='Region'), SeqFeature(FeatureLocation(ExactPosition(83), ExactPosition(174)), type='Region'), SeqFeature(FeatureLocation(ExactPosition(84), ExactPosition(170)), type='Region'), SeqFeature(

In [ ]:
#Note that a more typical use would be to save the sequence data to a local file, and then parse it with Bio.SeqIO. This can save you having to re-download the same file repeatedly while working on your script, and places less load on the NCBI’s servers. For example:

import os
from Bio import SeqIO
from Bio import Entrez
Entrez.email = "A.N.Other@example.com"  # Always tell NCBI who you are
filename = "EU490707.gbk"
if not os.path.isfile(filename):
    # Downloading...
    net_handle = Entrez.efetch(db="nucleotide", id="EU490707", rettype="gb", retmode="text")
    out_handle = open(filename, "w")
    out_handle.write(net_handle.read())
    out_handle.close()
    net_handle.close()
    print("Saved")

print("Parsing...")
record = SeqIO.read(filename, "genbank")
print(record)
#To get the output in XML format, which you can parse using the Bio.Entrez.read() function, use retmode="xml":

>>> from Bio import Entrez
>>> Entrez.email = "A.N.Other@example.com"  # Always tell NCBI who you are
>>> handle = Entrez.efetch(db="nucleotide", id="EU490707", retmode="xml")
>>> record = Entrez.read(handle)
>>> handle.close()
>>> record[0]["GBSeq_definition"]
'Selenipedium aequinoctiale maturase K (matK) gene, partial cds; chloroplast'
>>> record[0]["GBSeq_source"]
'chloroplast Selenipedium aequinoctiale'

5.1.2  Iterating over the records in a sequence file
In the above examples, we have usually used a for loop to iterate over all the records one by one. You can use the for loop with all sorts of Python objects (including lists, tuples and strings) which support the iteration interface.

The object returned by Bio.SeqIO is actually an iterator which returns SeqRecord objects. You get to see each record in turn, but once and only once. The plus point is that an iterator can save you memory when dealing with large files.

Instead of using a for loop, can also use the next() function on an iterator to step through the entries, like this:

from Bio import SeqIO
record_iterator = SeqIO.parse("ls_orchid.fasta", "fasta")

first_record = next(record_iterator)
print(first_record.id)
print(first_record.description)

second_record = next(record_iterator)
print(second_record.id)
print(second_record.description)
Note that if you try to use next() and there are no more results, you’ll get the special StopIteration exception.

One special case to consider is when your sequence files have multiple records, but you only want the first one. In this situation the following code is very concise:

from Bio import SeqIO
first_record = next(SeqIO.parse("ls_orchid.gbk", "genbank"))  
A word of warning here – using the next() function like this will silently ignore any additional records in the file. If your files have one and only one record, like some of the online examples later in this chapter, or a GenBank file for a single chromosome, then use the new Bio.SeqIO.read() function instead. This will check there are no extra unexpected records present.


#Extracting data
#The SeqRecord object and its annotation structures are described more fully in Chapter 4. As an example of how annotations are stored, we’ll look at the output from parsing the first record in the GenBank file ls_orchid.gbk.
http://biopython.org/DIST/docs/tutorial/Tutorial.html#chapter:Bio.SeqIO

In [29]:
from Bio import SeqIO
record_iterator = SeqIO.parse("ls_orchid.gbk", "genbank")
first_record = next(record_iterator)
print(first_record)

TypeError: 'NoneType' object is not callable

In [ ]:
That should give something like this:

ID: Z78533.1
Name: Z78533
Description: C.irapeanum 5.8S rRNA gene and ITS1 and ITS2 DNA.
Number of features: 5
/sequence_version=1
/source=Cypripedium irapeanum
/taxonomy=['Eukaryota', 'Viridiplantae', 'Streptophyta', ..., 'Cypripedium']
/keywords=['5.8S ribosomal RNA', '5.8S rRNA gene', ..., 'ITS1', 'ITS2']
/references=[...]
/accessions=['Z78533']
/data_file_division=PLN
/date=30-NOV-2006
/organism=Cypripedium irapeanum
/gi=2765658
Seq('CGTAACAAGGTTTCCGTAGGTGAACCTGCGGAAGGATCATTGATGAGACCGTGG...CGC', IUPACAmbiguousDNA())
This gives a human readable summary of most of the annotation data for the SeqRecord. For this example we’re going to use the .annotations attribute which is just a Python dictionary. The contents of this annotations dictionary were shown when we printed the record above. You can also print them out directly:

print(first_record.annotations)

In [ ]:
record_iterator = SeqIO.parse("ls_orchid.gbk", "genbank")
first_record = next(record_iterator)
print(first_record)

In [41]:
#I can do this for my SeqRecords from Entrez
#print(record.features)
print(len(record.features))
print(record.features[5:10])

55
[SeqFeature(FeatureLocation(ExactPosition(13), ExactPosition(14)), type='Region'), SeqFeature(FeatureLocation(ExactPosition(16), ExactPosition(17)), type='Region'), SeqFeature(FeatureLocation(ExactPosition(17), ExactPosition(59)), type='Region'), SeqFeature(FeatureLocation(ExactPosition(18), ExactPosition(60)), type='Region'), SeqFeature(FeatureLocation(ExactPosition(23), ExactPosition(24)), type='Region')]


In [44]:
#Print a specific feature within the record
f=record.features[14] #In this case PHD1
print(f)

type: Region
location: [292:343]
qualifiers:
    Key: experiment, Value: ['experimental evidence, no additional details recorded']
    Key: note, Value: ['PHD-type 1. {ECO:0000255|PROSITE-ProRule:PRU00146}.']
    Key: region_name, Value: ['Zinc finger region']



In [45]:
#extract the sequence of a specific feature within the record
f.extract(record.seq)

Seq('LYVCMFCGRGNNEDKLLLCDGCDDSYHTFCLLPPLPDVPKGDWRCPKCVAE', IUPACProtein())

In [72]:
#Can I extract the sequences of all domains/regions?
s = record.features
for f in s:
    if f.type == 'Region':
        #n=f.region_name #How do I get the names of the regions?
        r=f.extract(record.seq)
        #print(n)
        print(r)

MASVGPGGYAAEFVPPPECPVFEPSWEEFTDPLSFIGRIRPFAEKTGICKIRPPKDWQPPFACEVKTFRFTPRVQRLNELEAMTRVRLDFLDQLAKFWELQGSTLKIPVVERKILDLYALSKIVASKGGFEIVTKEKKWSKVGSRLGYLPGKGTGSLLKSHYERILYPYELFQSGVSLMGVQMPDLDLKEKVEAEVLSTDIQPSPERGTRMNIPPKRTRRVKSQSDSGEVNRNTELKKLQIFGAGPKVVGLAVGAKDKEDEVTRRRKVTNRSDAFNMQMRQRKGTLSVNFVDLYVCMFCGRGNNEDKLLLCDGCDDSYHTFCLLPPLPDVPKGDWRCPKCVAEECNKPREAFGFEQAVREYTLQSFGEMADNFKSDYFNMPVHMVPTELVEKEFWRLVSSIEEDVIVEYGADISSKDFGSGFPKKDGQRKMLPEEEEYALSGWNLNNMPVLEQSVLAHINVDISGMKVPWLYVGMCFSSFCWHIEDHWSYSINYLHWGEPKTWYGVPSHAAEQLEEVMRELAPELFESQPDLLHQLVTIMNPNVLMEHGVPVYRTNQCAGEFVVTFPRAYHSGFNQGYNFAEAVNFCTADWLPIGRQCVNHYRRLRRHCVFSHEELIFKMAADPECLDVGLAAMVCKELTLMTEEETRLRESVVQMGVVMSEEEVFELVPDDERQCSACRTTCFLSALTCSCNPERLVCLYHPTDLCSCPMQNKCLRYRYPLEDLPSLLYGVKVRAQSYDTWVNRVTEALSASFNHKKDLIELRVMLEDAEDRKYPENDLFRKLRDAVKEAETCGSVAQLLLSKKQKHRQSSDSGKTRTKLTVEELKAFVQQLVSLPCVISQTRQVKNLLDDVEEFHERAQEAMMDETPDSSKLQMLIDMGSSLYVELPELPRLKQELQQARWLDEVRLTLSDPQQVTLDVMKKLIDSGVGLAPHHAVEKAMAELQELLTVSERWEEKAKVCLQARPRHSMANLENIVNEAKNIPAFLPNVLSLKEALQK

In [ ]:
#To access data
Bio.SeqIO.index()

In [79]:
#Can I extract the sequences of all domains/regions?
s = record.features
for f in s:
    if f.type == 'Region':
        #re.finditer("PHD", f.qualifiers["note"], flags=0)
        i=(f.qualifiers)  

In [62]:
import re

In [65]:
re.finditer("ARID", f.qualifiers["region_name"], flags=0)

KeyError: 'region_name'

In [75]:
i=re.search("c", "abcdef") 

In [81]:
i.note

AttributeError: 'OrderedDict' object has no attribute 'note'